# L96 Ginelli Code Redevlop
Redevoloping code to work with observers. Will then allow me to checkpoint with ease.

## Working notes:

- Ginelli forward steps finished. Now need to think about how we calculate FTLEs and do backwards steps.

## Example Ginelli Forward Simulation

Here we write an example simulation of the forward phase of the Ginelli algorithm for two layer Lorenz 96.

In [1]:
# Standard Imports

import numpy as np
import time as tm
import pickle
from tqdm.notebook import tqdm
import xarray as xr
import sys
import os

In [2]:
# Lorenz 96 import
import l96tangent as l96t 

# Ginelli imports
import ginelli
from ginelli_observers import *

In [3]:
"""
Setup
-----------------------------------------
"""
# Memory parameters
dump_size = 5 # How many observation before dumping

# Time Parameter Choices
tau = 0.2

transient = 1.0 # adimensional time
ka = 5
kb = 15 # Number of BLV observations we want
kc = 12

# Runners
runner = l96t.Integrator(K=2, J=2)
ginelli_runner = ginelli.Forward(runner, tau) #relies on runner

# Observables
Rlooker = RMatrixObserver(ginelli_runner)
BLVlooker = BLVMatrixObserver(ginelli_runner)

ginelli.make_cupboard() 

Made directory ginelli.

Made directory ginelli/step2.

Made directory ginelli/step2/R.

Made directory ginelli/step2/BLV.

Made directory ginelli/step3.

Made directory ginelli/step4.

Made directory ginelli/step5.



In [4]:
"""
-----------------------------------------
Forward Part of Ginelli Algorithm
-----------------------------------------
"""
print('\nTransient beginning.\n')
runner.integrate(transient, noprog=False)
print('\nTransient finished. Beginning BLV convergence steps.\n')

# BLV Convergence steps

ginelli_runner.run(ka, noprog=False) 

# BLV and R Observation steps

print('\nBLV convergence finished. Beginning to observe BLVs.\n')
blocks = int(kb/dump_size) # How many times we dump
remainder = kb%dump_size # Rest of observations

for i in range(blocks):
    
    ginelli.make_observations(ginelli_runner, [Rlooker, BLVlooker], dump_size, 1) 
    # Observation frequency has to be 1 if we're reversing CLVs
    Rlooker.dump('ginelli/step2/R')
    BLVlooker.dump('ginelli/step2/BLV')

ginelli.make_observations(ginelli_runner, [Rlooker, BLVlooker], remainder, 1)
Rlooker.dump('ginelli/step2/R')
BLVlooker.dump('ginelli/step2/BLV')

# CLV Convergence Steps, just observe the Rs

print('\nBLV observations finished. CLV convergence beginning. Just observing Rs.\n')
blocks = int(kc/dump_size) 
remainder = kc%dump_size 

for i in range(blocks):
    
    ginelli.make_observations(ginelli_runner, [Rlooker], dump_size, 1)
    Rlooker.dump('ginelli/step3')

ginelli.make_observations(ginelli_runner, [Rlooker], remainder, 1)
Rlooker.dump('ginelli/step3')  
print('\nForward part all done :)')


Transient beginning.





Transient finished. Beginning BLV convergence steps.





BLV convergence finished. Beginning to observe BLVs.

Observations written to ginelli/step2/R/R1.nc. Erasing personal log.

Observations written to ginelli/step2/BLV/BLV1.nc. Erasing personal log.

Observations written to ginelli/step2/R/R2.nc. Erasing personal log.

Observations written to ginelli/step2/BLV/BLV2.nc. Erasing personal log.

Observations written to ginelli/step2/R/R3.nc. Erasing personal log.

Observations written to ginelli/step2/BLV/BLV3.nc. Erasing personal log.

I have no observations! :(
I have no observations! :(

BLV observations finished. CLV convergence beginning. Just observing Rs.

Observations written to ginelli/step3/R4.nc. Erasing personal log.

Observations written to ginelli/step3/R5.nc. Erasing personal log.

Observations written to ginelli/step3/R6.nc. Erasing personal log.


Forward part all done :)


## Backwards portion of Ginelli Algorithm Devlopment

In [5]:
"""
-------------------------------------------
Backward part of Ginelli Algorithm
-------------------------------------------
"""


"""Function for CLV Convergence Steps"""   

def block_squish_norm(R_history, A):
    """CLV convergence steps.
    Give a timeseries of R's, push A back through the timeseries, normalising as you go."""
    norms = np.linalg.norm(A, axis=0, ord=2)
    normedA = A/norms
    for R in np.flip(R_history.R, axis = 0):
        squishedA = np.linalg.solve(R, normedA)
        norms = np.linalg.norm(squishedA, axis=0, ord=2)
        normedA = squishedA/norms
    return normedA

In [6]:
""" 
Ginelli Algorithm Step 4, CLV Convergence Steps
"""
R_files = os.listdir('ginelli/step3')
R_files.sort(reverse=True) # Ensuring files are in the right order

A = np.identity(ginelli_runner.size)

for file in R_files:
    R_history = xr.open_dataset('ginelli/step3/' + file)
    A = block_squish_norm(R_history, A) # This A is one timestep ahead of the R that pushed it
    R_history.close()
    print(f'Pushed A through {file}. Overwriting A.npy.\n')
    np.save('ginelli/step4/A.npy', A)

Pushed A through R6.nc. Overwriting A.npy.

Pushed A through R5.nc. Overwriting A.npy.

Pushed A through R4.nc. Overwriting A.npy.



In [8]:
"""
Ginelli Algorithm Step 5, CLV Observation Steps
"""
# Sorting files we will be working with

def max_digit(strings):
    """Finds the max digit in a list of strings."""
    numbers = []
    for string in strings:
        for s in string:
            if s.isdigit():
                numbers.append(int(s))
    return max(numbers)

R_files = os.listdir('ginelli/step2/R')
R_files.sort(reverse=True) 

BLV_files = os.listdir('ginelli/step2/BLV')
BLV_files.sort(reverse=True)

# Setting up observable storage

parameters = ginelli_runner.parameter_dict.copy().update({'transient':transient,'ka':ka, 'kb':kb})
LyapunovLooker = LyapunovObserver(parameters, max_digit(BLV_files))


for [rfile, bfile] in zip(R_files, BLV_files): # Loop over files that were dumped
    R_history = xr.open_dataset('ginelli/step2/R/' + rfile) # Dataset
    BLV_history = xr.open_dataset('ginelli/step2/BLV/' + bfile)
    
    Rs, BLVs = np.flip(R_history.R, axis = 0), np.flip(BLV_history.BLV, axis = 0) # Times series reversed
    
    for R, BLV in zip(Rs, BLVs): # Calculating CLVs
        
        # CLV Calculation
        CLV = np.matmul(BLV.values, A)
        
        # FTCLE Calculation
        squishedA = np.linalg.solve(R, A)
        norms = np.linalg.norm(squishedA, axis=0, ord=2)
        ftcle = - np.log(norms)/(tau)
        A = squishedA/norms
        
        # FTBLE Calculation
        ftble = np.log(np.diag(R))/(tau)
        
        # Making observation
        LyapunovLooker.look(R.time, CLV, BLV, ftcle, ftble)
        
        
    LyapunovLooker.dump('ginelli/step5')
    R_history.close()
    BLV_history.close()


Observations written to ginelli/step5/LyapObs3.nc. Erasing personal log.

Observations written to ginelli/step5/LyapObs2.nc. Erasing personal log.

Observations written to ginelli/step5/LyapObs1.nc. Erasing personal log.

